In [1]:
#import re
# Affiche le répertoire courant
#current_directory = os.getcwd()
#print("Répertoire courant :", current_directory)
#import os
#os.chdir('/Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing')


Répertoire courant : /Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing


In [1]:
import sys
sys.path.append('../')
import glob 

import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Initialiser tqdm pour pandas
from common.utils import translate_to_english
from common.utils import clean_text
from common.utils import extract_salary_info
from common.utils import get_infos_location, load_location_cache, save_location_cache

import pickle
from datetime import datetime
#pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
#pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des cellules


In [52]:
import glob 
raw_path = '../outputs/raw/'

jobs_adzuna_pkl = glob.glob(f'{raw_path}*jobs_adzuna*.pkl')
jobs_adzuna = pd.concat([pd.read_pickle(file) for file in jobs_adzuna_pkl], axis=0)

jobs_themuse_pkl = glob.glob(f'{raw_path}*jobs_muse*.pkl')
jobs_themuse = pd.concat([pd.read_pickle(file) for file in jobs_themuse_pkl], axis=0)

jobs_linkedin_pkl = glob.glob(f'{raw_path}*jobs_linkedin*.pkl')
jobs_linkedin = pd.concat([pd.read_pickle(file) for file in jobs_linkedin_pkl], axis=0)

jobs_wttj_pkl  = glob.glob(f'{raw_path}*jobs_wttj*.pkl')
jobs_wttj = pd.concat([pd.read_pickle(file) for file in jobs_wttj_pkl], axis=0)

print('taille jobs_adzuna:',jobs_adzuna.shape)
print('taille jobs_linkedin:',jobs_linkedin.shape)
print('taille jobs_themuse:',jobs_themuse.shape)
print('taille jobs_wttj:',jobs_wttj.shape)


taille jobs_adzuna: (389950, 24)
taille jobs_linkedin: (7270, 15)
taille jobs_themuse: (1980, 16)
taille jobs_wttj: (13994, 21)


In [4]:
jobs_adzuna_selected = jobs_adzuna[['title','description','contract_type', 'contract_time',
'company.display_name', 'created','location.display_name','category.label', 'salary_max' ,'salary_min', #'location.area',
'redirect_url','source', 'country']].rename(columns={'title': 'job_title', 'description': 'job_description',
'company.display_name': 'company_name','created': 'publication_datetime','location.display_name': 'location','category.label': 'category',
'redirect_url': 'offer_link', 'country':'country_adzuna'})
#'location.area': 'locations_company',

jobs_adzuna_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_adzuna_selected['publication_time'] = jobs_adzuna_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
#jobs_adzuna_selected['country']='US'
jobs_adzuna_selected['industry']='not available'
#jobs_adzuna_selected['currency']='dollars'
jobs_adzuna_selected['levels']='not available'


#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].astype(str)
#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].str.replace(r'[\[\]]', '', regex=True)
jobs_adzuna_selected.head()

print( 'taille jobadzuna avant dedoubloonnage', jobs_adzuna_selected.shape)
#companies['contents'] = companies['description'].apply(clean_text)
jobs_adzuna_selected=jobs_adzuna_selected.drop_duplicates()
print( 'taille jobadzuna apres dedoubloonnage', jobs_adzuna_selected.shape)


taille jobadzuna avant dedoubloonnage (389950, 17)
taille jobadzuna apres dedoubloonnage (136944, 17)


In [5]:
#locat='Fort Douglas, Salt Lake County'
#get_infos_location(locat)

jobs_adzuna_selected.location.nunique() #16329
jobs_adzuna_selected.location.count() 

np.int64(136944)

Cache chargé.


In [ ]:
cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
jobs_adzuna_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_adzuna_selected['location'].progress_apply(get_infos_location)

print( 'taille jobadzuna apres dedoubloonnage', jobs_adzuna_selected.shape)
save_location_cache(cache_path)


In [48]:
jobs_adzuna_selected.columns

Index(['job_title', 'job_description', 'contract_type', 'contract_time',
       'company_name', 'publication_datetime', 'location', 'category',
       'salary_max', 'salary_min', 'offer_link', 'source', 'country_adzuna',
       'publication_date', 'publication_time', 'industry', 'levels', 'country',
       'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name'],
      dtype='object')

In [50]:
#location_cache['Phoenix, eThekwini']
#correction_location('Phoenix, eThekwini')

In [39]:
'''from geopy.geocoders import Nominatim
import pycountry
cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
def  correction_location(location_name):
    try:
        geolocator = Nominatim(user_agent="my_geolocation_app_v1.0", timeout=5)  # Initialisation
        
        # Géocodage de la localisation
        location_obj = geolocator.geocode(location_name)
        if not location_obj:
            return pd.Series(['Not found'] * 7)
        
        # Extraction des coordonnées
        latitude = str(location_obj.latitude)
        longitude = str(location_obj.longitude)
        
        # Reverse géocodage pour obtenir les détails de l'adresse
        location_reverse = geolocator.reverse(f"{latitude},{longitude}")
        address = location_reverse.raw.get('address', {})
        
        # Extraction des informations d'adresse
        country = address.get('country', 'Unknown')
        city = address.get('city', address.get('town', 'Unknown'))
        state = address.get('state', 'Unknown')
        postcode = address.get('postcode', 'Unknown')
        
        # Obtention des informations sur la devise
        country_info = pycountry.countries.get(name=country)
        if country_info:
            currency = pycountry.currencies.get(numeric=country_info.numeric)
            currency_name = currency.name if currency else 'Unknown'
        else:
            currency_name = 'Unknown'
        
        # Création du résultat à renvoyer sous forme de liste
        result = [country, state, city, postcode, latitude, longitude, currency_name]
        
        # Mise à jour du cache avec les informations
        location_cache[location_name] = result
        
        return pd.Series(result)
    except Exception as e:
        # Gestion des erreurs générales
        print(f"Error for location '{location_name}': {e}")
        location_cache[location_name] = ['Not found'] * 7  # Mise à jour du cache avec un résultat d'erreur
        return pd.Series(['Not found'] * 7)
    
import time

for l in location_list:
    correction_location(l)
    time.sleep(2)
save_location_cache(cache_path)
'''

Cache chargé.


In [51]:
jobs_adzuna_selected.head()

,job_title,job_description,contract_type,contract_time,company_name,publication_datetime,location,category,salary_max,salary_min,...,publication_time,industry,levels,country,state,city,postcode,latitude,longitude,currency_name
0,Registered Nurse,About the role: We now seek additional Nurses ...,NaN,full_time,Akari,2024-10-28T09:31:29Z,"Deckham, Gateshead",Healthcare & Nursing Jobs,38400.0,38400.0,...,09:31:29,not available,not available,United Kingdom,England,Gateshead,NE9 5LB,54.9468789,-1.5909164,Pound Sterling
1,Bar and Waiting Staff,Bar and Waiting Staff £12 per hour South Wales...,NaN,part_time,Catering Services International,2024-11-11T11:33:56Z,"Cardiff, Cardiff County",Hospitality & Catering Jobs,23040.0,23040.0,...,11:33:56,not available,not available,United Kingdom,Cymru / Wales,Cardiff,CF14 3QU,51.50316895,-3.178576542915388,Pound Sterling
2,Flexible Unqualified Nursery Practitioner,Come and join Partou UK for the next step in y...,permanent,part_time,Just Childcare,2024-11-05T09:01:16Z,"Knowle, Bristol",Teaching Jobs,22060.0,12384.0,...,09:01:16,not available,not available,United Kingdom,England,Bristol,BS4 1BH,51.4285779,-2.5779992,Pound Sterling
3,Registered Nurse,About the role: Please note to be considered f...,NaN,full_time,Akari,2024-10-25T10:29:16Z,"Jesmond, Newcastle Upon Tyne",Healthcare & Nursing Jobs,38400.0,38400.0,...,10:29:16,not available,not available,United Kingdom,England,Newcastle upon Tyne,NE2 2DX,54.9914956,-1.6057608,Pound Sterling
4,Registered Nurse,Please note to be considered for the Register...,NaN,full_time,Akari,2024-10-18T10:20:48Z,"Aycliffe, Newton Aycliffe",Healthcare & Nursing Jobs,38400.0,38400.0,...,10:20:48,not available,not available,United Kingdom,England,Newton Aycliffe,DL5 7JQ,54.613739,-1.5897097,Pound Sterling


In [55]:
jobs_themuse.head() 


,contents,name,type,publication_date,short_name,model_type,id,locations,categories,tags,refs.landing_page,company.id,company.short_name,company.name,levels.name,levels.short_name
0,<p><strong>Gemeinsam machen wir den Unterschie...,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,external,2023-01-24T00:42:10Z,freileitungsmonteur-wmd-fahrleitungs-und-strom...,jobs,10536487,"[Bad Mergentheim, Germany]",Computer and IT,[],https://www.themuse.com/jobs/siemens/freileitu...,11913,siemens,Siemens,Mid Level,mid
1,<p><strong>Mechanical Assembler</strong><br><b...,Mechanical Assembler - 1st Shift,external,2023-04-26T23:41:35Z,mechanical-assembler-1st-shift-0a7d96,jobs,11259124,"[Clayton, NC]",Software Engineering,[],https://www.themuse.com/jobs/siemens/mechanica...,11913,siemens,Siemens,Senior Level,senior
2,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician,external,2024-06-06T12:25:46Z,pharmacy-technician-4dfa8b,jobs,14569255,"[Rockville, MD]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Senior Level,senior
3,<p><strong>Who designs your future? You do.</s...,Sr. Systems Specialist - Smart Buildings / Aut...,external,2023-05-03T23:45:16Z,sr-systems-specialist-smart-buildings-automati...,jobs,11034620,"[Waterloo, IA]",Design and UX,[],https://www.themuse.com/jobs/siemens/sr-system...,11913,siemens,Siemens,Senior Level,senior
4,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician Front End - Full Time,external,2024-06-06T12:33:58Z,pharmacy-technician-front-end-full-time-416939,jobs,14444118,"[Edison, NJ]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Mid Level,mid


In [58]:
jobs_theMuse_selected = jobs_themuse[['name','contents','type', 'company.name',
'publication_date', 'locations','categories','refs.landing_page','levels.name']].rename(columns={'name': 'job_title', 'contents': 'job_description',
'type': 'contract_type','company.name': 'company_name','publication_date': 'publication_datetime','locations': 'location',
'categories': 'category','refs.landing_page': 'offer_link','levels.name':'levels'})

jobs_theMuse_selected['publication_date'] = pd.to_datetime(jobs_theMuse_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_theMuse_selected['publication_time'] = jobs_theMuse_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())

#jobs_theMuse_selected['country']='a travailler'
jobs_theMuse_selected['industry']='not available'
#jobs_theMuse_selected['currency']='not available'
jobs_theMuse_selected['contract_time']='not available'
jobs_theMuse_selected['salary_max']=0
jobs_theMuse_selected['salary_min']=0

jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].astype(str)
jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_theMuse_selected['locations_company']=jobs_theMuse_selected['location']

jobs_theMuse_selected['job_description'] = jobs_theMuse_selected['job_description'].apply(clean_text)

jobs_theMuse_selected["source"] = "TheMuse"
jobs_theMuse_selected=jobs_theMuse_selected.drop_duplicates()
print( 'taille jobs_theMuse apres dedoubloonnage', jobs_theMuse_selected.shape)

jobs_theMuse_selected.head()


taille jobs_theMuse apres dedoubloonnage (1980, 16)


,job_title,job_description,contract_type,company_name,publication_datetime,location,category,offer_link,levels,publication_date,publication_time,industry,contract_time,salary_max,salary_min,source
0,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,<p><strong>Gemeinsam machen wir den Unterschie...,external,Siemens,2023-01-24T00:42:10Z,"'Bad Mergentheim, Germany'",Computer and IT,https://www.themuse.com/jobs/siemens/freileitu...,Mid Level,2023-01-24,00:42:10,not available,not available,0,0,TheMuse
1,Mechanical Assembler - 1st Shift,<p><strong>Mechanical Assembler</strong><br><b...,external,Siemens,2023-04-26T23:41:35Z,"'Clayton, NC'",Software Engineering,https://www.themuse.com/jobs/siemens/mechanica...,Senior Level,2023-04-26,23:41:35,not available,not available,0,0,TheMuse
2,Pharmacy Technician,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:25:46Z,"'Rockville, MD'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Senior Level,2024-06-06,12:25:46,not available,not available,0,0,TheMuse
3,Sr. Systems Specialist - Smart Buildings / Aut...,<p><strong>Who designs your future? You do.</s...,external,Siemens,2023-05-03T23:45:16Z,"'Waterloo, IA'",Design and UX,https://www.themuse.com/jobs/siemens/sr-system...,Senior Level,2023-05-03,23:45:16,not available,not available,0,0,TheMuse
4,Pharmacy Technician Front End - Full Time,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:33:58Z,"'Edison, NJ'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Mid Level,2024-06-06,12:33:58,not available,not available,0,0,TheMuse


In [59]:
cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
jobs_theMuse_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_theMuse_selected['location'].progress_apply(get_infos_location)

print( 'taille jobs_theMuse_selected', jobs_theMuse_selected.shape)
save_location_cache(cache_path)

Cache chargé.


100%|██████████| 1980/1980 [49:20<00:00,  1.50s/it] 

taille jobs_theMuse_selected (1980, 23)


In [64]:
#len(location_cache)#12884
jobs_theMuse_selected.columns

Index(['job_title', 'job_description', 'contract_type', 'company_name',
       'publication_datetime', 'location', 'category', 'offer_link', 'levels',
       'publication_date', 'publication_time', 'industry', 'contract_time',
       'salary_max', 'salary_min', 'source', 'country', 'state', 'city',
       'postcode', 'latitude', 'longitude', 'currency_name'],
      dtype='object')

In [69]:
jobs_adzuna_selected.columns

Index(['job_title', 'job_description', 'contract_type', 'contract_time',
       'company_name', 'publication_datetime', 'location', 'category',
       'salary_max', 'salary_min', 'offer_link', 'source', 'country_adzuna',
       'publication_date', 'publication_time', 'industry', 'levels', 'country',
       'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name'],
      dtype='object')

In [70]:

#df[['currency', 'min_salary', 'max_salary']] = df['salary'].apply(extract_salary_info)
jobs_wttj.head()

,title,company,type_company,site_company,type_contract,salary,contents,date_public,monthsOfExperience,employmentType,...,qualifications,validThrough,logo,addressCountry,postalCode,addressLocality,streetAddress,industry,addressRegion,link_job
0,Manager Conseil en transformation data (H/F),iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
1,Consultant.e senior Stratégie & Conseil,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
2,Consultant.e Senior Supply chain,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
3,Consultant.e Contract Management,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,...,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...


In [74]:
jobs_wttj.columns

Index(['title', 'company', 'type_company', 'site_company', 'type_contract',
       'salary', 'contents', 'date_public', 'monthsOfExperience',
       'employmentType', 'remote', 'qualifications', 'validThrough', 'logo',
       'addressCountry', 'postalCode', 'addressLocality', 'streetAddress',
       'industry', 'addressRegion', 'link_job'],
      dtype='object')

In [75]:
jobs_wttj_selected = jobs_wttj[['title', 'company', 'type_contract', 'contents', 'date_public',
       'employmentType','addressCountry', 'addressLocality',  'industry', 'link_job', 'salary', 'postalCode', 'streetAddress']].rename(
columns={'title': 'job_title', 'contents': 'job_description',
'employmentType': 'contract_type','company': 'company_name','date_public': 'publication_datetime','addressLocality': 'location',
' addressCountry': 'country_wttj','link_job': 'offer_link','streetAddress': 'streetAdress_wttj','postalCode': 'postalCode_wttj'})



jobs_wttj_selected['publication_date'] = pd.to_datetime(jobs_wttj_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_wttj_selected['publication_time'] = jobs_wttj_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
jobs_wttj_selected['industry']='not available'
jobs_wttj_selected['contract_time']='not available'
jobs_wttj_selected['category']='not available'

jobs_wttj_selected[['currency_wttj', 'salary_min', 'salary_max']] = jobs_wttj_selected['salary'].apply(extract_salary_info)
jobs_wttj_selected = jobs_wttj_selected.drop('salary', axis=1)

jobs_wttj_selected['location'] = jobs_wttj_selected['location'].astype(str)
jobs_wttj_selected['location'] = jobs_wttj_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_wttj_selected['locations_company']=jobs_wttj_selected['location']

jobs_wttj_selected['job_description'] = jobs_wttj_selected['job_description'].apply(clean_text)
jobs_wttj_selected["levels"] = "not available"

jobs_wttj_selected["source"] = "Wttj"
jobs_wttj_selected=jobs_wttj_selected.drop_duplicates()


jobs_wttj_selected.head()

,job_title,company_name,type_contract,job_description,publication_datetime,contract_type,addressCountry,location,industry,offer_link,...,streetAdress_wttj,publication_date,publication_time,contract_time,category,currency_wttj,salary_min,salary_max,levels,source
0,Manager Conseil en transformation data (H/F),iQo,Permanent contract,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,...,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
1,Consultant.e senior Stratégie & Conseil,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,...,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
2,Consultant.e Senior Supply chain,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,...,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
3,Consultant.e Contract Management,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,...,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,...,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,not available,Not,0.0,0.0,not available,Wttj


In [76]:
cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
jobs_wttj_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_wttj_selected['streetAdress_wttj'].progress_apply(get_infos_location)

print( 'taille jobs_theMuse_selected', jobs_wttj_selected.shape)
save_location_cache(cache_path)

Cache chargé.


100%|██████████| 13994/13994 [3:02:50<00:00,  1.28it/s]  

taille jobs_theMuse_selected (13994, 28)


In [13]:
#jobs_linkedin.link.unique()
jobs_linkedin.head()

,title,company,location,link,salary,salary_min,salary_max,seniority_level,employment_type,job_function,job_industry,posted_time,company_link,source,ingest_date
0,Healthcare Scheduler - Fully Remote,Vigilance Health,United States,https://www.linkedin.com/jobs/view/healthcare-...,$18.00 - $19.00,$18.00,$19.00,Entry level,Full-time,Health Care Provider,Hospitals and Health Care,2024-11-02,https://www.linkedin.com/company/vigilance-hea...,linkedin,2024-11-02 19:54:25.139908
1,Marketing Assistant,PyjamaHR,"New York, NY",https://www.linkedin.com/jobs/view/marketing-a...,,,,Entry level,Full-time,Administrative,Professional Training and Coaching,2024-11-02,https://www.linkedin.com/company/computercore?...,linkedin,2024-11-02 19:54:25.139908
2,Online Proofreader,IEM,United States,https://www.linkedin.com/jobs/view/online-proo...,,,,Not Applicable,Full-time,Sales and Business Development,"Retail Apparel and Fashion, Design Services, a...",2024-11-02,https://www.linkedin.com/company/beyond-yoga?t...,linkedin,2024-11-02 19:54:25.139908
3,Bagger,Jungle Jim's International Market,"Fairfield, OH",https://www.linkedin.com/jobs/view/bagger-at-j...,,,,Entry level,Part-time,Information Technology,Human Resources Services,2024-11-02,https://www.linkedin.com/company/timelyfind?tr...,linkedin,2024-11-02 19:54:25.139908
4,Data Entry Operator,Computer CORE (Community OutReach and Education),United States,https://www.linkedin.com/jobs/view/data-entry-...,$45.00 - $50.00,$45.00,$50.00,Entry level,Full-time,Other,Retail,2024-11-02,https://www.linkedin.com/company/jungle-jim's-...,linkedin,2024-11-02 19:54:25.139908


In [77]:
jobs_linkedin_selected = jobs_linkedin[['title','employment_type', 'company','salary_min','salary_max',
'posted_time','location','job_industry','link','source','seniority_level','job_function']].rename(columns={'title': 'job_title', 'employment_type': 'contract_time',
'company': 'company_name','posted_time': 'publication_datetime','job_industry': 'industry','link': 'offer_link','seniority_level':'levels','job_function':'category'})

#jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_linkedin_selected['contents']='not available'
#jobs_linkedin_selected['currency']='dollars'
jobs_linkedin_selected['publication_time'] =None

jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_linkedin_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_linkedin_selected['contract_type']='not available'

#jobs_linkedin_selected['locations_company']='not available'

#jobs_linkedin_selected['job_description'] = jobs_linkedin_selected['job_description'].apply(clean_text)
jobs_linkedin_selected.head()

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,levels,category,contents,publication_time,publication_date,contract_type
0,Storyboard Artist - Unannounced Disney Junior ...,Full-time,The Walt Disney Company,"$65,000.00","$95,000.00",2024-11-09,"Glendale, CA",Entertainment Providers,https://www.linkedin.com/jobs/view/storyboard-...,linkedin,Associate,"Design, Art/Creative, and Information Technology",not available,None,2024-11-09,not available
1,Private Equity Associate,Full-time,"HRPivot, Inc",,,2024-11-09,"Chicago, IL",Internet Publishing,https://www.linkedin.com/jobs/view/private-equ...,linkedin,Entry level,Management and Manufacturing,not available,None,2024-11-09,not available
2,Photographer,Full-time,CoCreativ,,,2024-11-09,"New York, NY",Software Development,https://www.linkedin.com/jobs/view/photographe...,linkedin,Mid-Senior level,Other,not available,None,2024-11-09,not available
3,"Coordinator, Events (Hulu)",Full-time,The Walt Disney Company,,,2024-11-09,"Santa Monica, CA",Government Administration and Strategic Manage...,https://www.linkedin.com/jobs/view/coordinator...,linkedin,Not Applicable,Sales and Business Development,not available,None,2024-11-09,not available
4,Remote Sports Events Coordinator,Full-time,LC LUXE,$30.00,$35.00,2024-11-09,"Los Angeles, CA",IT Services and IT Consulting,https://www.linkedin.com/jobs/view/remote-spor...,linkedin,Mid-Senior level,Human Resources,not available,None,2024-11-09,not available


12884

In [79]:
cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
jobs_linkedin_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_linkedin_selected['location'].progress_apply(get_infos_location)

print( 'taille jobs_theMuse_selected', jobs_linkedin_selected.shape)
save_location_cache(cache_path)

Cache chargé.


100%|██████████| 7270/7270 [32:25<00:00,  3.74it/s]  

taille jobs_theMuse_selected (7270, 23)


In [82]:
jobs_linkedin_selected[['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name']] = jobs_linkedin_selected['location'].progress_apply(get_infos_location)


12884

In [83]:
jobs1 = pd.concat([jobs_linkedin_selected, jobs_theMuse_selected], axis=0)
jobs2 = pd.concat([jobs1, jobs_adzuna_selected], axis=0)
jobs_final = pd.concat([jobs2, jobs_wttj_selected], axis=0)
jobs_final=jobs_final.drop_duplicates()
#jobs = pd.concat([jobs2, jobs_wttj_selected], axis=0)

shape=jobs_final.shape
print(f"Taille du dataframe fusionné : {shape}")
#Taille du dataframe fusionné : (16269, 21)


Taille du dataframe fusionné : (160188, 30)


In [84]:
jobs_final.columns

Index(['job_title', 'contract_time', 'company_name', 'salary_min',
       'salary_max', 'publication_datetime', 'location', 'industry',
       'offer_link', 'source', 'levels', 'category', 'contents',
       'publication_time', 'publication_date', 'contract_type', 'country',
       'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name',
       'job_description', 'country_adzuna', 'type_contract', 'addressCountry',
       'postalCode_wttj', 'streetAdress_wttj', 'currency_wttj'],
      dtype='object')

In [85]:
jobs_final.head()

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,...,latitude,longitude,currency_name,job_description,country_adzuna,type_contract,addressCountry,postalCode_wttj,streetAdress_wttj,currency_wttj
0,Storyboard Artist - Unannounced Disney Junior ...,Full-time,The Walt Disney Company,"$65,000.00","$95,000.00",2024-11-09,"Glendale, CA",Entertainment Providers,https://www.linkedin.com/jobs/view/storyboard-...,linkedin,...,34.1469416,-118.2478471,US Dollar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Private Equity Associate,Full-time,"HRPivot, Inc",,,2024-11-09,"Chicago, IL",Internet Publishing,https://www.linkedin.com/jobs/view/private-equ...,linkedin,...,41.8755616,-87.6244212,US Dollar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Photographer,Full-time,CoCreativ,,,2024-11-09,"New York, NY",Software Development,https://www.linkedin.com/jobs/view/photographe...,linkedin,...,40.7127281,-74.0060152,US Dollar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Coordinator, Events (Hulu)",Full-time,The Walt Disney Company,,,2024-11-09,"Santa Monica, CA",Government Administration and Strategic Manage...,https://www.linkedin.com/jobs/view/coordinator...,linkedin,...,34.0194704,-118.491227,US Dollar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Remote Sports Events Coordinator,Full-time,LC LUXE,$30.00,$35.00,2024-11-09,"Los Angeles, CA",IT Services and IT Consulting,https://www.linkedin.com/jobs/view/remote-spor...,linkedin,...,34.0536909,-118.242766,US Dollar,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
#pd.crosstab(jobs_final.country_adzuna, jobs_final.country)
jobs_final.country_adzuna.value_counts

<bound method IndexOpsMixin.value_counts of 0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
13989    NaN
13990    NaN
13991    NaN
13992    NaN
13993    NaN
Name: country_adzuna, Length: 160188, dtype: object>

In [16]:
jobs_final.head()
jobs_final['contents'] = jobs_final['contents'].progress_apply(translate_to_english)


100%|██████████| 25571/25571 [00:09<00:00, 2825.46it/s]


In [17]:
jobs_final.to_pickle('../outputs/final/jobs.pkl')


In [ ]:

#jupyter nbconvert --to script process_jobs.ipynb